In [1]:
!pip install -q darts


^C
Traceback (most recent call last):
  File "/Users/reecepalmer/.pyenv/versions/3.10.6/envs/bakery/bin/pip", line 8, in <module>
    sys.exit(main())
  File "/Users/reecepalmer/.pyenv/versions/3.10.6/envs/bakery/lib/python3.10/site-packages/pip/_internal/cli/main.py", line 79, in main
    return command.main(cmd_args)
  File "/Users/reecepalmer/.pyenv/versions/3.10.6/envs/bakery/lib/python3.10/site-packages/pip/_internal/cli/base_command.py", line 101, in main
    return self._main(args)
  File "/Users/reecepalmer/.pyenv/versions/3.10.6/envs/bakery/lib/python3.10/site-packages/pip/_internal/cli/base_command.py", line 119, in _main
    level_number = setup_logging(
  File "/Users/reecepalmer/.pyenv/versions/3.10.6/envs/bakery/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 275, in setup_logging
    logging.config.dictConfig(
  File "/Users/reecepalmer/.pyenv/versions/3.10.6/lib/python3.10/logging/config.py", line 810, in dictConfig
    dictConfigClass(config).configu

In [ ]:
import pandas as pd
import numpy as np

from darts import TimeSeries
from darts.models import LightGBMModel
from darts.metrics import mse
from darts.dataprocessing.transformers import Scaler


/Users/reecepalmer/.pyenv/versions/3.10.6/envs/bakery/lib/python3.10/site-packages/statsforecast/core.py:25: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [ ]:
df = pd.DataFrame(pd.read_csv('../raw_data/hourly_final_dataset.csv'))
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.set_index('timestamp')


In [ ]:
target = df[['traditional_baguette', 'croissant', 'coupe', 'pain_au_chocolat', 'baguette', 'banette', 'cereal_baguette']]
past_cov = df[['temperature_2m (°C)', 'relative_humidity_2m (%)', 'apparent_temperature (°C)', 'precipitation (mm)', 'rain (mm)', 'wind_speed_10m (km/h)', 'wind_speed_100m (km/h)', 'day_of_week_sin', 'day_of_week_cos', 'month_sin', 'month_cos', 'isHoliday']]
future_cov = df[['isHoliday']]


In [ ]:
y_train = target.loc[:'2021-12-31']
y_val = target.loc['2021-12-31':'2022-01-07']
past_cov_train = past_cov.loc[:'2021-12-31']
past_cov_val = past_cov.loc['2021-12-25':'2022-01-07']


In [ ]:
df_series = TimeSeries.from_dataframe(df)
y_train_series = TimeSeries.from_dataframe(y_train)
y_val_series = TimeSeries.from_dataframe(y_val)
past_cov_train_series = TimeSeries.from_dataframe(past_cov_train)
past_cov_val_series = TimeSeries.from_dataframe(past_cov_val)


In [ ]:
lags = 7 * 24
lags_past_covariates = 7 * 24
lags_future_covariates = 7 * 24
output_chunk_length = 7 * 24
horizon = 168


In [ ]:
def eval_global_model(train_series, val_series, model):

  preds = model.predict(n = horizon, past_covariates = past_cov_val_series)

  score = mse(val_series, preds)
  return score


In [ ]:
lgbm = LightGBMModel(
    lags=lags,
    lags_past_covariates=lags_past_covariates,
    output_chunk_length=output_chunk_length,
    horizon=horizon,
    num_leaves=31,
    learning_rate=0.05,
    n_estimators=100,
    verbose=-1
)

lgbm.fit(y_train_series, past_covariates=past_cov_train_series)


[LightGBM] [Warning] Unknown parameter: horizon
[LightGBM] [Warning] Unknown parameter: horizon
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.128481 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 254688
[LightGBM] [Info] Number of data points in the train set: 8425, number of used features: 3192
[LightGBM] [Info] Start training from score 7.546469
[LightGBM] [Warning] Unknown parameter: horizon
[LightGBM] [Warning] Unknown parameter: horizon
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.145594 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 254688
[LightGBM] [Info] Number of data points in the train set: 8425, number of used features: 3192
[LightGBM] [Info] Start training from score 1.890326
[LightGBM] [Warning] Unknown parameter: horizon
[LightGBM] [Warning] Unknown parameter: horizon
[LightGBM] [Info] Auto

In [ ]:
scoring = eval_global_model(y_train_series, y_val_series, lgbm)
scoring


37.537639342397824